In [9]:
from networkx.readwrite import json_graph
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.algorithms.community import greedy_modularity_communities
import itertools
import community
import os, os.path
import fnmatch
import random as rand
import json
import matplotlib.colors as colors
import matplotlib.cm as cmx


In [10]:
with open('config.json') as handle:
    config = json.load(handle)
book = config['book_name']
window_size = config['temporal_window_size'] 
dir_path = './%s/part'% book
image_path = "./community_images/temporal_graphs/%s/" %book

In [11]:
temporal_year_list = [[1885,1894],[1895,1904],[1905,1915],[1915,1924]]
with open('json_files/' + book + '/name_to_fullname.json') as handle:
    name_to_fullname = json.load(handle)
community_list = {}
community_list['person'] = {}
community_list['place'] = {}

In [12]:
for year_range in temporal_year_list:
    start_year = year_range[0]
    end_year = year_range[1]
    for graph_type in ["place","person"]:
        year_range = str(start_year) + '-' + str(end_year)
        community_list[graph_type][year_range] = {}
        """
        with open('json_files/' + book + '/' + graph_type + '_entity_year_dict.json') as handle:
            node_year_list = json.load(handle)
        """

        #convert list of years to string so that it can be easily stored
        """
        node_year_string = {}
        for node in node_year_list.keys():
            node_year_string[node]  = '_'.join(node_year_list[node])
        """
        graph_file_path = 'temporal_graphs/' + book +'/graph_' + graph_type + '_startyear_' + str(start_year) + '_endyear_' + str(end_year) + '.xml'
        G = nx.read_graphml(graph_file_path)
        # python-louvain

        part_graph = community.best_partition(G)
        community_dict = {}
        year_dict = {}
        for node in G.nodes():
            if node not in year_dict:
                year_dict[node] = []
            community_dict[node] = part_graph.get(node)

        communities = [part_graph.get(node) for node in G.nodes()]
        plt.figure(figsize=(30, 30))
        nx.draw_networkx(G, cmap = plt.get_cmap('Pastel1'), node_color = communities, with_labels=True,
                         pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())),
                         iterations=20), font_weight='bold', font_family = 'serif',edge_color = '#acff9d',node_size = 600)
        plt.savefig(image_path + graph_type + "_windowsize_" + str(window_size) + "_louvian.png")
        plt.close()
        pos = nx.spring_layout(G, k=0.3*1/np.sqrt(len(G.nodes())))

        nx.set_node_attributes(G, name='community', values=community_dict)
        for entity,community_no in community_dict.items():
            community_str = "community_" +str(community_no)
            if community_str not in community_list[graph_type][year_range].keys():
                community_list[graph_type][year_range][community_str] = []
            if entity in name_to_fullname.keys():
                community_list[graph_type][year_range][community_str].append(name_to_fullname[entity])
            else:
                community_list[graph_type][year_range][community_str].append(entity)
            
        print(community_dict)
        # need to figure out how to pass it as list
        #nx.set_node_attributes(G, name='years', values=node_year_string)
        nx.write_graphml(G, 'temporal_graphs/' + book + '/graph_' + graph_type  + '_startyear_' + str(start_year) + '_endyear_' + str(end_year) + '_community.xml')

{'Bhavnagar': 0, 'Paris': 1, 'Malta': 0, 'France': 1, 'Richmond': 1, 'Johannesburg': 1, 'Junagadh': 0, 'Lahore': 2, 'North': 2, 'Rajkot': 2, 'Durban': 1, 'Delhi': 2, 'Natal': 1, 'Germany': 1, 'Porbandar': 2, 'London': 0, 'America': 1, 'Ahmedabad': 0, 'Brighton': 1, 'Karachi': 2, 'Southampton': 0, 'Wellington': 0, 'Netley': 1, 'Rome': 2, 'Aden': 1, 'Pretoria': 2, 'Charlestown': 2, 'Portsmouth': 1, 'England': 2, 'South Africa': 0, 'India': 0, 'Ventnor': 1, 'Mozambique': 2, 'Zanzibar': 2}
{'Mavji Dave': 0, 'Cecilia Roberts': 1, 'Malaviyaji': 2, 'Jehangir Petit': 2, 'Muhammad Kasam Kamruddin': 2, 'Rajchandra': 2, 'Dada Abdulla': 0, 'Dawud': 2, 'Balasundaram': 2, 'Lady Tata': 2, 'Dalpatram': 0, 'Akho': 1, 'Maffey': 2, 'Addison': 1, 'John Burns': 1, 'Narsinhram': 2, 'Ladha Maharaj': 0, 'Krause': 0, 'Frederic': 0, 'Escombe': 2, 'Allinson': 0, 'Dadabhai Naoroji': 0, 'Anna Kingsford': 0, 'Rangasami Padiachi': 2, 'Narmadashanker': 2, 'Tulasidas': 0, 'Pearson': 2, 'Lord Elgin': 2, 'George Muller'

In [13]:
with open('json_files/%s/community_people_list.json' % book, 'w') as handle:
    json.dump(community_list["person"],handle,indent = 4)

with open('json_files/%s/community_place_list.json' % book, 'w') as handle:
    json.dump(community_list['place'],handle,indent = 4)